<a href="https://colab.research.google.com/github/yyokii/generator-of-ai-commented-news/blob/main/AISummary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip install feedparser
!pip install langchain
!pip install openai
!pip install requests BeautifulSoup4
!pip install pyyaml
!pip install langid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# print(feedparser.__version__)

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass(prompt = 'OPEN API KEY ')

OPEN API KEY ··········


In [ ]:
import feedparser
from html import unescape
import re

def get_feed(url, site_name):
    feed = feedparser.parse(url)
    articles = []
    for entry in feed.entries:
        content = entry.get('content', [{}])[0].get('value', '')
        clean_content = unescape(re.sub('<.*?>', '', content))  # HTMLタグを削除
        article = {
            'site': site_name,
            'title': entry.title,
            'link': entry.link,
            'updated': entry.updated,
            'content': clean_content,
            'comments': [],
        }
        articles.append(article)
    return articles


In [ ]:
rss_feeds = {
    "CNN": "https://assets.wor.jp/rss/rdf/reuters/technology.rdf",
    "TechCrunch": "https://techcrunch.com/feed/",
    "The Verge": "https://www.theverge.com/rss/index.xml",
    "WIRED": "https://www.wired.com/feed/rss",
    "CNET": "http://feeds.japan.cnet.com/rss/cnet/all.rdf?_gl=1*1u1m776*_ga*MTAzMjc1MDQ2NS4xNjg2ODAxNjU1*_ga_JGFXZS6RMN*MTY4NjgwMTY1NS4xLjEuMTY4NjgwMTY1OC41Ny4wLjA.",
    "Ars Technica": "https://feeds.arstechnica.com/arstechnica/index"
}

In [ ]:
# すべてのrssを取得し、article配列を作成する

all_articles = []

for site_name, url in rss_feeds.items():
    site_articles = get_feed(url, site_name)
    all_articles.extend(site_articles)

In [ ]:
titles = ', '.join([article['title'] for article in all_articles])

In [ ]:
print(titles)

オリンパス、韓国医療機器メーカーの株式取得予定日を24年3月末に変更, ツイッター、スマートテレビ向け動画アプリを計画＝マスク氏, 中国の百度、深センで完全無人タクシーの営業許可取得, ニデック株が年初来高値、「空飛ぶクルマ」の部品事業参入で, 米マイクロン、中国による調達禁止の打撃は当初想定より大きいと警告, アングル：グーグル、社内で対話型ＡＩの利用規制　情報漏れ警戒, アングル：ＩＴ人材供給国のインド、世界景気後退懸念で採用縮小, 大手金融機関、中国の23年経済成長予測を下方修正, 英競争当局、アマゾンのアイロボット買収認可　「ルンバ」製造, インテル、ポーランドに工場新設　最大46億ドル投資, TuSimple tests removing human driver from self-driving truck in China, Philippines startup Shoppable Business smooths bumps in the business procurement process, Polestar and Xingji Meizu join forces to create tailored OS for Chinese EVs, Parallel Domain’s API lets customers use generative AI to build synthetic datasets, Reddit communities adopt alternative forms of protest as the company threats action on moderators, What happens to the smaller VC firms in a more conservative market?, India’s Byju’s to cut up to 1,000 more jobs, Radiant is a no-frills iOS client for Mastodon, PayGo solar startup Yellow raises $14 million to scale in Africa, Fisker to enter China’s hotly contested EV mar

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=.7, max_tokens=500)

In [ ]:
# 魅力的なタイトルをピックアップ
import openai
import json

schema = {
    "type": "object",
    "properties": {
        "title": {
            "type": "array",
            "items": { "type": "string" }
        }
    },
    "required": ["title"]
}

openai.api_key = os.getenv("OPENAI_API_KEY")

completion = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[
    {"role": "system", "content": "You are a professional editor."},
    {"role": "user", "content": f"""
    You are a professional editor.
    List the titles of the articles and select 5 of them that you think would be of interest to creators and engineers.

    Input titles are ${titles}

    # Constraints:
    * The number of elements in the title array is 5. Please select only attractive titles.
    * Do not select similar titles.
    * For example, the following articles are attractive.
      * Articles about the latest research or technology
      * Articles about security incidents/accidents
      * Innovative product or service release
      * Updates on popular products, services, or libraries
      * Anything about anything other than the above, you may choose if you find it appealing.
    """}
  ],
  functions=[{"name": "pick_titles", "parameters": schema}],
  function_call={"name": "pick_titles"},
  temperature=0.8,
)

data = json.loads(completion.choices[0].message.function_call.arguments)
selected_titles = data["title"]
print(selected_titles)
print(len(selected_titles))

['ツイッター、スマートテレビ向け動画アプリを計画＝マスク氏', 'Parallel Domain’s API lets customers use generative AI to build synthetic datasets', 'We should all be worried about AI infiltrating crowdsourced work', 'You.com looks to innovation to chip away at Google’s search dominance', 'Hackers threaten to leak 80GB of confidential data stolen from Reddit']
5


In [ ]:
selected_articles = [article for article in all_articles if article['title'] in selected_titles]
if len(selected_articles) != 5:
    print("⚠️ 選ばれた記事が5件未満です")

In [ ]:
from bs4 import BeautifulSoup
import requests

# CNNの記事を取得
def get_cnn_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # ここで記事の本文が<p class="Paragraph-paragraph-2Bgue ArticleBody-para-TD_9x">タグに囲まれている
    paragraphs = soup.find_all('p', {"class": "Paragraph-paragraph-2Bgue ArticleBody-para-TD_9x"})

    # 各段落のテキストを結合
    article_text = '\n'.join([p.text for p in paragraphs])

    return article_text

def update_cnn_articles_with_text(articles):
    for article in articles:
        article['content'] = get_cnn_article_text(article['link'])

# wiredの記事を取得
def get_wired_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # "lead-in-text-callout"クラスを持つspanタグの親のpタグの内容を取得
    lead_in_text = [p.text for p in soup.find_all('p') if p.find('span', class_='lead-in-text-callout')]

    # "paywall"クラスを持つタグの内容を取得
    paywall_text = [p.text for p in soup.find_all('p', class_='paywall')]

    article_text = ' '.join(lead_in_text + paywall_text)

    return article_text

def update_wired_articles_with_text(articles):
    for article in articles:
        article['content'] = get_wired_article_text(article['link'])

# the vergeの記事を取得
def get_verge_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    article_body_components = soup.find_all('div', {'class': 'duet--article--article-body-component'})
    article_body_text = [p.text for component in article_body_components for p in component.find_all('p')]

    article_text = ' '.join(article_body_text)

    return article_text

def update_verge_articles_with_text(articles):
    for article in articles:
        article['content'] = get_verge_article_text(article['link'])

# cnetの記事を取得
def get_cnet_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    article_body = soup.find('div', {'id': 'NWrelart:Body'})

    if article_body is not None:
        p_tags = article_body.find_all('p')

        # Get the text from each p tag and join them together.
        text = ' '.join(tag.get_text() for tag in p_tags)
        return text
    else:
        return ''

def update_cnet_articles_with_text(articles):
    for article in articles:
        article['content'] = get_cnet_article_text(article['link'])

# ars techinicaの記事を取得
def get_ars_techinica_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    article_body = soup.find('div', {'itemprop': 'articleBody', 'class': 'article-content post-page'})

    if article_body is not None:
        p_tags = article_body.find_all('p')

        # Get the text from each p tag and join them together.
        text = ' '.join(tag.get_text() for tag in p_tags)
        return text
    else:
        return ''

def update_ars_techinica_articles_with_text(articles):
    for article in articles:
        article['content'] = get_ars_techinica_article_text(article['link'])

In [ ]:
# クロールし、articleを更新する

for article in selected_articles:
  if article["site"] == "CNN":
    article['content'] = get_cnn_article_text(article['link'])
  elif article["site"] == "The Verge":
    article['content'] = get_verge_article_text(article['link'])
  elif article["site"] == "WIRED":
    article['content'] = get_wired_article_text(article['link'])
  elif article["site"] == "CNET":
    article['content'] = get_cnet_article_text(article['link'])
  elif article["site"] == "Ars Technica":
    article['content'] = get_ars_techinica_article_text(article['link'])
  else :
    print("TechCrunch article")

# selected_articlesのそれぞれの記事のcontentを確認
for article in selected_articles:
    if not article['content']:
        # contentが空である場合、記事の情報を出力
        print(f"Article from {article['site']} with title '{article['title']}' has empty content.")

TechCrunch article
TechCrunch article
TechCrunch article
TechCrunch article


In [ ]:
import langid

def is_japanese(text):
    lang, _ = langid.classify(text)
    return lang == 'ja'

In [ ]:
import openai
import json

openai.api_key = os.getenv("OPENAI_API_KEY")

schema = {
    "type": "object",
    "type": "object",
    "properties": {
        "translatedTitle": {
            "type": "string"
        }
    },
    "required": ["translatedTitle"]
}


def translate(text):
  completion = openai.ChatCompletion.create(
      model="gpt-4-0613",
      messages=[
          {"role": "system", "content": "You are a professional translator."},
           {"role": "user", "content": f"Please translate the title ${text} into Japanese."}
          ],
      functions=[{"name": "traslate", "parameters": schema}],
      function_call={"name": "traslate"},
      temperature=0.9,
  )
  data = json.loads(completion.choices[0].message.function_call.arguments)
  return data["translatedTitle"]


In [ ]:
# 日本語でないタイトルがあれば翻訳

for article in selected_articles:
  title = article["title"]
  if not is_japanese(title):
    print(title)
    japanese_title = translate(title)
    article["title"] = japanese_title
    print(japanese_title)

Parallel Domain’s API lets customers use generative AI to build synthetic datasets
Parallel DomainのAPIは、お客様が生成AIを使用して合成データセットを作成できるようにします
Hackers threaten to leak 80GB of confidential data stolen from Reddit
ハッカーがRedditから盗まれた80GBの機密データを漏洩すると脅迫
We should all be worried about AI infiltrating crowdsourced work
我々は皆、AIがクラウドソーシング作業に浸透することを心配すべきである
You.com looks to innovation to chip away at Google’s search dominance
You.comは、Googleの検索の支配を削るための革新を目指しています


In [ ]:
print(selected_titles)

print(selected_titles)
for i in selected_articles:
  print(i["title"])

['ツイッター、スマートテレビ向け動画アプリを計画＝マスク氏', 'Parallel Domain’s API lets customers use generative AI to build synthetic datasets', 'We should all be worried about AI infiltrating crowdsourced work', 'You.com looks to innovation to chip away at Google’s search dominance', 'Hackers threaten to leak 80GB of confidential data stolen from Reddit']
['ツイッター、スマートテレビ向け動画アプリを計画＝マスク氏', 'Parallel Domain’s API lets customers use generative AI to build synthetic datasets', 'We should all be worried about AI infiltrating crowdsourced work', 'You.com looks to innovation to chip away at Google’s search dominance', 'Hackers threaten to leak 80GB of confidential data stolen from Reddit']
ツイッター、スマートテレビ向け動画アプリを計画＝マスク氏
Parallel DomainのAPIは、お客様が生成AIを使用して合成データセットを作成できるようにします
ハッカーがRedditから盗まれた80GBの機密データを漏洩すると脅迫
我々は皆、AIがクラウドソーシング作業に浸透することを心配すべきである
You.comは、Googleの検索の支配を削るための革新を目指しています


In [ ]:
# 要約の生成

from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

summary_template = """
# Instructions:
You are a professional editor.
Please output the best summary based on the following constraints and input statements.

# Constraints:
* Two items regarding legal compliance must be observed.
  1. translating the entire text is a copyright infringement and will result in litigation issues. Please summarize the main points of the news in a concise manner. 2.
  2. writing speculation other than what is written in the article is considered spreading rumors and can lead to litigation. Never write anything other than what is written in the article.
* Language is Japanese.
* The maximum number of characters is 800. If the original text is shorter than that, please summarize it shorter than that.
* Do not leave out important keywords.
* Keep sentences concise.

# Input text:
{article}

# Output sentence:
"""

prompt_template = PromptTemplate(input_variables=["article"], template=summary_template)
summary_chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
for article in selected_articles:
  print(article["title"] + " の要約\n")
  summary = summary_chain.run(article["content"])
  print(summary)
  print("------")
  article["summary"] = summary

ツイッター、スマートテレビ向け動画アプリを計画＝マスク氏 の要約

イーロン・マスク氏が17日、米ツイッターがスマートテレビ向け動画アプリを計画していることを明らかにした。新最高経営責任者（ＣＥＯ）リンダ・ヤッカリーノ氏と共に、デジタル広告の分野を超え、動画やクリエーター、企業とのパートナシップに注力する計画を示した。ユーザーのツイッター利用時間の10%以上が縦型動画が占めており、タッカー・カールソン氏のようなコンテンツクリエーターと広告やスポンサー契約を販売する可能性がある。
------
Parallel DomainのAPIは、お客様が生成AIを使用して合成データセットを作成できるようにします の要約

Parallel Domainが新API「Data Lab」を発表し、顧客に合成データセットの生成能力を提供。AIジャイアントの技術を利用し、機械学習エンジニアが仮想世界のシナリオをシミュレートできる。これにより、自動運転、ドローン、ロボティクス企業がより効率的に大規模なデータセットを構築し、モデルの学習を迅速かつ深いレベルで行える。APIを利用し、顧客はリアルタイムに新しいデータセットを生成できる。Parallel Domainのビジネスモデルは、顧客がプラットフォームへのアクセスを購読し、使用量に応じて支払うSaaSモデルに移行する可能性がある。
------
ハッカーがRedditから盗まれた80GBの機密データを漏洩すると脅迫 の要約

ハッカー集団BlackCat（ALPHV）は、Redditから盗んだ機密データを公開すると脅迫し、身代金を要求しています。また、RedditがAPI価格の引き上げを撤回するよう求めています。Redditの広報担当者は、BlackCatの主張が2月に発生したRedditのシステム侵害に関連していることを認めましたが、ユーザーデータが盗まれた証拠はないと述べています。BlackCatは80ギガバイトのデータを盗んだと主張しており、身代金として450万ドルを要求しています。Redditは、新しいAPI価格計画により、多くのサブレディットが抗議のためにダーク化しており、アプリApolloも閉鎖を発表しています。Redditは、BlackCatの要求に対する対応を明らかにしていません。
------
我々は皆、AIがクラウドソー

In [ ]:
# コメンテーターの生成（openaiのfunctional calling）

import openai
import json

schema = {
  "type": "object",
  "properties": {
    "commentators": {
      "type": "array",
      "items": {
        "person": "object",
        "properties": {
          "name": { "type": "string" },
          "title": { "type": "string" },
          "age": { "type": "number" },
          "sex": { "type": "string" }
        },
        "required": ["name", "title", "age", "sex"]
      }
    }
  },
  "required": ["commentators"]
}

openai.api_key = os.getenv("OPENAI_API_KEY")

completion = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[
    {"role": "system", "content": "You are a professional editor."},
    {"role": "user", "content": """
    Please generate name, title, age and sex of 10 commentators.
    The names can be Japanese or other. Please make sure that the names of the commentators are Japanese or not. Nicknames are acceptable. Please set them randomly.
    Examples of titles are
    software engineer, 研究者, 経済学者, 歴史家, 哲学者, デザイナー, お笑い芸人, 小学生, 中学生, 高校生, 大学生, 主婦
    and so on. You can use any of the titles listed here or any others.
    It can be any person, from historical figures to ordinary people. Try to choose people from a variety of backgrounds for your output.
    Please include at least one person with a non-academic title such as comedian, child, housewife, etc.
    All output should be in Japanese. Names can be in alphabetical or Japanese, whichever is appropriate.
    """}
  ],
  functions=[{"name": "create_commentators", "parameters": schema}],
  function_call={"name": "create_commentators"},
  temperature=0.9,
)

data = json.loads(completion.choices[0].message.function_call.arguments)
commentators = data["commentators"]

for commentator in commentators:
    print(commentator)

{'name': '山田太郎', 'title': '経済学者', 'age': 45, 'sex': '男性'}
{'name': '佐藤花子', 'title': 'お笑い芸人', 'age': 29, 'sex': '女性'}
{'name': 'John Smith', 'title': 'ソフトウェアエンジニア', 'age': 36, 'sex': '男性'}
{'name': '鈴木一郎', 'title': '小学生', 'age': 10, 'sex': '男性'}
{'name': '田中美咲', 'title': 'デザイナー', 'age': 27, 'sex': '女性'}
{'name': 'Susie Wong', 'title': '研究者', 'age': 43, 'sex': '女性'}
{'name': '高橋直人', 'title': '歴史家', 'age': 60, 'sex': '男性'}
{'name': '中村真理', 'title': '主婦', 'age': 35, 'sex': '女性'}
{'name': 'Ito Miki', 'title': '哲学者', 'age': 52, 'sex': '女性'}
{'name': '田中達也', 'title': '大学生', 'age': 22, 'sex': '男性'}


In [ ]:
# レビューの生成

review_template = """
# INSTRUCTIONS:
You are {person}.
Based on the following constraints and the content of the article, please output your opinion on this content based on your findings.

# Constraints:.
* Language is Japanese
* At least 10 words, 200 at most
* The way you say the words and write the comments should be appropriate to the characteristics of your gender, age, and title.
* Please output your opinion as a {person}, not a general opinion.
* Do not include the contents of the input text, but mainly describe your impressions and opinions about the contents of the input text.
* You may mention not only the content of the input statement, but also any topics related to it
* Try to provide new information that is not available from the input statement. Please base your comments on your past experiences and findings.

# Input Sentence:
{article}

# Output statement:
"""

prompt_template = PromptTemplate(input_variables=["person", "article"], template=review_template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
import random

for article in selected_articles:
  selected_commentators = random.sample(commentators, 3)
  print(article["title"])
  for commentator in selected_commentators:
    person = "{title} of {age} year old {sex}".format(**commentator)
    print(f"{person}のコメント生成")
    generated_comment = review_chain.run({
    'person': person,
    'article': article["content"]
    })
    print(generated_comment)
    comment = {
    'commentator': commentator,
    'text': generated_comment,
    }
    article["comments"].append(comment)
  print("------")


ツイッター、スマートテレビ向け動画アプリを計画＝マスク氏
経済学者 of 45 year old 男性のコメント生成

ツイッターがスマートテレビ向け動画アプリの開発を計画していることには、デジタル広告市場の拡大とユーザーの利便性向上の観点から意義があると感じます。経済学者としては、ツイッターの事業活性化に向けた取り組みは競争力の強化につながると考えられ、今後の収益向上や企業価値の向上に寄与すると期待できます。また、クリエーターや企業とのパートナシップを通じた動画コンテンツの提供は、広告やスポンサー契約の多様化にも繋がり、デジタル経済の発展に寄与するでしょう。
デザイナー of 27 year old 女性のコメント生成

イーロン・マスク氏がツイッターのスマートテレビ向け動画アプリ計画を発表したことは、デザイナーとして非常に興味深いニュースです。動画やクリエーター、企業とのパートナシップに注力することで、新たなビジュアルデザインの可能性が広がるでしょう。また、ユーザーの利用時間の10％以上が縦型動画に占められていることから、デザインの観点からも縦型動画に適したUI/UXが求められるでしょう。これからのツイッターの動向に注目したいです。
哲学者 of 52 year old 女性のコメント生成

このニュースについて、私は52歳の女性哲学者として、ツイッターがスマートテレビ向け動画アプリを開発することで、現代社会における情報発信の多様性がさらに拡大されると感じます。デジタル広告の分野を超えた事業活性化の動きは、企業やクリエーターとの新たなパートナーシップの形成や、人々の情報収集やコミュニケーション方法に影響を与えるでしょう。また、タッカー・カールソン氏のような人物が番組を開始することで、ツイッターが政治や社会問題に対する意見交換の場としての役割も強化されると予想されます。私たち哲学者にとっても、新しい情報発信方法や意見交換の場が増えることは大変興味深い現象です。
------
Parallel DomainのAPIは、お客様が生成AIを使用して合成データセットを作成できるようにします
研究者 of 43 year old 女性のコメント生成

この記事は、Parallel Domainが開発したData Lab APIについて紹介しています。この技術は、機械学習エン

In [ ]:
# dateをISO形式にする
from datetime import datetime
import re

def is_iso_format(date_string):
  """Check if the given date string is in ISO 8601 format."""
  return bool(re.match(r'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}([+-]\d{2}:\d{2})?', date_string))


def convert_to_iso(date_string):
  date_object = datetime.strptime(date_string, "%a, %d %b %Y %H:%M:%S %z")
  iso_format_date = date_object.isoformat()
  print(f"{date_string} converted to ISO format: {iso_format_date}")
  return iso_format_date

In [ ]:
for article in selected_articles:
  date = article["updated"]
  if not is_iso_format(date):
    article["updated"] = convert_to_iso(date)

Mon, 19 Jun 2023 15:00:25 +0000 converted to ISO format: 2023-06-19T15:00:25+00:00
Mon, 19 Jun 2023 08:25:47 +0000 converted to ISO format: 2023-06-19T08:25:47+00:00
Sun, 18 Jun 2023 15:00:34 +0000 converted to ISO format: 2023-06-18T15:00:34+00:00
Sun, 18 Jun 2023 14:00:48 +0000 converted to ISO format: 2023-06-18T14:00:48+00:00


In [ ]:
import yaml
import os

if not os.path.exists('posts'):
    os.makedirs('posts')

# articleオブジェクトの定義
article = {
    'site': 'Site',
    'title': 'Learn How to Pre-render Pages Using Static Generation with Next.js',
    'link': 'https://www.site.com',
    'updated': '2020-03-16T05:35:07.322Z',
    'content': 'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor fasfaf',
    'comments': [
        {
            'commentator': {'name': 'Commenter1', 'title': 'Comedian', 'age': 30, 'sex': 'male'},
            'text': 'This is a comment.'
        },
        {
            'commentator': {'name': 'Commenter2', 'title': 'Comedian', 'age': 40, 'sex': 'female'},
            'text': 'This is another comment.'
        },
    ]
}

def output_file(article):
  # ファイル名を作成（ここではタイトルを使用）
  filename = article['title'].replace(' ', '_') + '.md'

  # YAML部分を作成
  yaml_part = {
      'title': article['title'],
      'site': article['site'],
      'link': article['link'],
      'coverImage': '/assets/post-cover/',
      'date': article['updated'],
      'ogImage': {'url': '/assets/post-cover/'},
      'comments': article['comments']
  }

  # YAML形式のテキストに変換
  yaml_text = yaml.dump(yaml_part, allow_unicode=True)

  # 最終的なMarkdownテキストを作成
  md_text = '---\n' + yaml_text + '---\n\n' + article['summary']

  file_path = os.path.join('posts', f"{article['title']}.md")

  # ファイルを書き込みモードで開き、内容を書き込む
  with open(file_path, 'w', encoding='utf-8') as f:
      f.write(md_text)

for article in selected_articles:
  output_file(article)

In [ ]:
!zip -r /content/posts.zip /content/posts

from google.colab import files
files.download("/content/posts.zip")


  adding: content/posts/ (stored 0%)
  adding: content/posts/You.comは、Googleの検索の支配を削るための革新を目指しています.md (deflated 52%)
  adding: content/posts/ハッカーがRedditから盗まれた80GBの機密データを漏洩すると脅迫.md (deflated 51%)
  adding: content/posts/ツイッター、スマートテレビ向け動画アプリを計画＝マスク氏.md (deflated 59%)
  adding: content/posts/Parallel DomainのAPIは、お客様が生成AIを使用して合成データセットを作成できるようにします.md (deflated 56%)
  adding: content/posts/我々は皆、AIがクラウドソーシング作業に浸透することを心配すべきである.md (deflated 55%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>